In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv(r"data\unlabelled\santacruz_cleaned.csv")
df

In [ ]:
normals = {
    1: 30.4, 2: 31.1, 3: 32.6, 4: 33.0, 5: 33.3, 6: 32.0,
    7: 29.9, 8: 29.5, 9: 30.2, 10: 33.1, 11: 33.4, 12: 31.9
}
df.insert(loc=8, column='Normal_Temp', value=df['MN'].map(normals))

In [ ]:
df

In [ ]:
df.to_csv(r"data\unlabelled\santacuz_with_normal.csv")

In [ ]:
daily_max = df.groupby(['YEAR', 'MN', 'DT'])['DBT'].max().reset_index()
thresholds = {month: normal + 4.5 for month, normal in normals.items()}

# Function to determine heatwave label
def is_heatwave(dbt, month):
    if dbt >= 37.0 or dbt >= thresholds[month]:
        return 1
    return 0

# Merge daily max DBT back to original dataframe
df = df.merge(daily_max[['YEAR', 'MN', 'DT', 'DBT']], on=['YEAR', 'MN', 'DT'], suffixes=('', '_max'))
df['Heatwave'] = df.apply(lambda row: is_heatwave(row['DBT_max'], row['MN']), axis=1)
df = df.drop(columns=['DBT_max'])  # Remove temporary max column
print(thresholds)

In [ ]:
df

In [ ]:
heatwave_counts = df['Heatwave'].value_counts()
heatwave_counts

In [ ]:
if os.path.exists(r"data\labelled\santacruz_labelled.csv"):
    print("Labelled processed data exists")
else:
    df.to_csv(r"data\labelled\santacruz_labelled.csv", index= False)
    print("Labelled processed data created")

In [ ]:
df['DATE'] = pd.to_datetime(df[['YEAR', 'MN', 'DT']].rename(columns={'MN': 'month', 'DT': 'day'}))
df.set_index('DATE', inplace=True)
df

In [ ]:
df.drop(columns=['INDEX', 'YEAR', 'MN', 'DT', 'HR'], inplace=True)


In [ ]:
daily_df = df.resample('D').mean()


In [ ]:
if os.path.exists(r"data\labelled\santacruz_labelled_daily.csv"):
    print("Labelled processed data exists")
else:
    daily_df.to_csv(r"data\labelled\santacruz_labelled_daily.csv", index= False)
    print("Labelled processed data created")

In [ ]:
df_daily = pd.read_csv(r"data\labelled\santacruz_labelled_daily.csv")

In [ ]:
df_daily

In [ ]:
df_daily.isnull().sum()

In [31]:
df_filled = pd.read_csv(r"data\unlabelled\filled_santacruz_daily.csv", index_col=False)
df_filled

,DATE,DPT,WBT,DBT,Normal_Temp,SLP,MSLP,RH,DD,FFF,RF
0,01-01-2010,16.0875,19.775,25.725,30.4,1009.5000,1011.1625,57.000,12.500,4.250,0.0
1,02-01-2010,18.2000,21.375,26.975,30.4,1011.0000,1012.6875,61.125,6.750,3.250,0.0
2,03-01-2010,19.7375,22.025,26.300,30.4,1012.3500,1014.0500,67.875,21.875,5.250,0.0
3,04-01-2010,18.0250,20.450,24.575,30.4,1011.5750,1013.2500,67.875,25.250,6.250,0.0
4,05-01-2010,18.5500,20.575,24.175,30.4,1009.2375,1010.9125,72.875,11.875,3.750,0.0
...,...,...,...,...,...,...,...,...,...,...,...
5443,26-11-2024,14.7625,27.400,24.675,33.4,0.0000,0.0000,57.500,0.625,0.750,0.0
5444,27-11-2024,13.9750,27.400,25.375,33.4,0.0000,0.0000,51.625,1.250,1.875,0.0
5445,28-11-2024,12.6875,27.400,24.850,33.4,0.0000,0.0000,49.125,1.250,1.500,0.0
5446,29-11-2024,12.7625,27.400,24.500,33.4,0.0000,0.0000,51.375,7.000,4.500,0.0


In [32]:
threshold = {temp: temp + 4.5 for temp in df_filled['Normal_Temp']}

In [33]:
df_filled.insert(loc=5, column='Threshold_Temp', value=df_filled['Normal_Temp'].map(threshold))


In [ ]:
df_filled

,DATE,DPT,WBT,DBT,Normal_Temp,Threshold_Temp,SLP,MSLP,RH,DD,FFF,RF
0,01-01-2010,16.0875,19.775,25.725,30.4,34.9,1009.5000,1011.1625,57.000,12.500,4.250,0.0
1,02-01-2010,18.2000,21.375,26.975,30.4,34.9,1011.0000,1012.6875,61.125,6.750,3.250,0.0
2,03-01-2010,19.7375,22.025,26.300,30.4,34.9,1012.3500,1014.0500,67.875,21.875,5.250,0.0
3,04-01-2010,18.0250,20.450,24.575,30.4,34.9,1011.5750,1013.2500,67.875,25.250,6.250,0.0
4,05-01-2010,18.5500,20.575,24.175,30.4,34.9,1009.2375,1010.9125,72.875,11.875,3.750,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5443,26-11-2024,14.7625,27.400,24.675,33.4,37.9,0.0000,0.0000,57.500,0.625,0.750,0.0
5444,27-11-2024,13.9750,27.400,25.375,33.4,37.9,0.0000,0.0000,51.625,1.250,1.875,0.0
5445,28-11-2024,12.6875,27.400,24.850,33.4,37.9,0.0000,0.0000,49.125,1.250,1.500,0.0
5446,29-11-2024,12.7625,27.400,24.500,33.4,37.9,0.0000,0.0000,51.375,7.000,4.500,0.0


In [47]:
df_filled.to_csv(r"data\unlabelled\filled_santacruz_daily_threahold.csv", index=False)

In [48]:
df_filled['Heatwave'] = df_filled.apply(lambda row: 1 if (row['DBT'] > 37 or row['DBT'] > row['Threshold_Temp']) else 0, axis=1)


In [49]:
df_filled

,DATE,DPT,WBT,DBT,Normal_Temp,Threshold_Temp,SLP,MSLP,RH,DD,FFF,RF,Heatwave
0,01-01-2010,16.0875,19.775,25.725,30.4,34.9,1009.5000,1011.1625,57.000,12.500,4.250,0.0,0
1,02-01-2010,18.2000,21.375,26.975,30.4,34.9,1011.0000,1012.6875,61.125,6.750,3.250,0.0,0
2,03-01-2010,19.7375,22.025,26.300,30.4,34.9,1012.3500,1014.0500,67.875,21.875,5.250,0.0,0
3,04-01-2010,18.0250,20.450,24.575,30.4,34.9,1011.5750,1013.2500,67.875,25.250,6.250,0.0,0
4,05-01-2010,18.5500,20.575,24.175,30.4,34.9,1009.2375,1010.9125,72.875,11.875,3.750,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5443,26-11-2024,14.7625,27.400,24.675,33.4,37.9,0.0000,0.0000,57.500,0.625,0.750,0.0,0
5444,27-11-2024,13.9750,27.400,25.375,33.4,37.9,0.0000,0.0000,51.625,1.250,1.875,0.0,0
5445,28-11-2024,12.6875,27.400,24.850,33.4,37.9,0.0000,0.0000,49.125,1.250,1.500,0.0,0
5446,29-11-2024,12.7625,27.400,24.500,33.4,37.9,0.0000,0.0000,51.375,7.000,4.500,0.0,0


In [50]:
heatwave_counts = df_filled['Heatwave'].value_counts()
heatwave_counts

Heatwave
0    5448
Name: count, dtype: int64

In [ ]:
if os.path.exists(r"data\labelled\santacruz_daily_filled_labelled.csv"):
    print("Labelled processed data exists")
else:
    df.to_csv(r"data\labelled\santacruz_daily_filled_labelled.csv")
    print("Labelled processed data created")

In [ ]:
new_df = pd.read_csv(r"data\labelled\santacruz_daily_filled_labelled.csv")
new_df

In [ ]:
new_df['heatwave_days'] = 0
consecutive_days = (new_df['Heatwave'] == 1) & (new_df['Heatwave'].shift(1) == 1)

# Assign 1 to the identified consecutive heatwave days
new_df.loc[consecutive_days | consecutive_days.shift(-1, fill_value=False), 'heatwave_days'] = 1

In [ ]:
new_df

In [ ]:
print(new_df.columns.tolist())


In [ ]:
new_df['DATE'][new_df['DATE'].duplicated()].unique()


In [ ]:
df.to_csv(r"data\labelled\santacruz_labelled_days.csv")